# HW4 
# Task №1

# Preporation
For solving task 1 and 2 we will define function for lazy FCA classification. 

This function will use compute intersect function for getting intersect between train row and row we want to classify. And function for computing extent based on the intersect. Also added function for computing intervals as argument with default value for future tasks 4 and 5. 

And as I working on Big Homework with FCALC Lazy Classifier, I decided to add it's classification to tasks 1-3 for fun (As interval functions there can not be modified for tasks 4 and 5).

In [5]:
import pandas as pd
import numpy as np

import fcalc

from sklearn.metrics import accuracy_score, f1_score

def compute_itersect(data_row, classification_row, interval_constructor=None):
    intersection = []

    if len(data_row) - len(classification_row) != 1 and len(data_row) != len(classification_row):
        raise Exception

    for i in range(len(classification_row)):
        if type(classification_row[i]) is str:
            if data_row[i] != classification_row[i]:
                intersection.append('*')
            else:
                intersection.append(classification_row[i])
        else:
            if interval_constructor is None:
                intersection.append((min(data_row[i], classification_row[i]), max(data_row[i], classification_row[i])))
            else:
                intersection.append(interval_constructor(data_row[i], classification_row[i]))

    return np.array(intersection, dtype=object)

def compute_extent(data, target, pattern):
    extent = []
    
    if len(data) == 0:
        return extent

    if data.shape[1] - pattern.shape[0] != 1 and data.shape[1] != pattern.shape[0]:
        raise Exception 

    for row_index in range(data.shape[0]):
        row = data.iloc[row_index]
        row_target = target.iloc[row_index]
        is_fit = True
        for i in range(len(pattern)):
            if type(row[i]) is str:
                if pattern[i] != '*' and row[i] != pattern[i]:
                    is_fit = False
                    break
            else:
                if row[i] < pattern[i][0] or row[i] > pattern[i][1]:
                    is_fit = False
                    break
        
        if is_fit:
            extent.append((row_index, row_target))

    return extent

def classificator(X_train, y_train, X_test, verbose=0, interval_constructor=None):
    answers = []
    for test_row in X_test.iloc:
        is_classified = False
        for train_row in X_train.iloc:
            pattern = compute_itersect(train_row.to_list(), test_row.to_list(), interval_constructor=interval_constructor)
            extent = compute_extent(X_train, y_train, pattern)
            if verbose:
                print(pattern)
                print(extent)
            if len(extent) != 0 and all(extent[0][1] == x[1] for x in extent):
                answers.append(extent[0][1])
                is_classified = True
                break
        if not is_classified:
            answers.append('U')
    return answers

# Subtask 1

In [6]:
data = pd.read_csv("Wheel_Chains_Dataset.csv", sep=';')
data

,System,mount,price,con,snow,ice,dur,Accegrade
0,SK,F,206,1.9,1.4,1.8,2.7,F
1,SRK,F or R,520,2.1,0.8,3.8,2.3,F
2,SK,F,160,1.7,1.9,1.6,3.7,F
3,SK,F,213,1.7,2.0,2.4,3.4,F
4,SMS,F or R,598,1.6,2.4,2.7,2.8,F
5,SK,F,109,2.0,1.9,2.4,3.7,F
6,SRK,F or R,325,2.0,2.1,3.2,2.8,F
7,SMS,F or R,498,1.5,3.3,3.5,2.0,T
8,SRK,F or R,396,2.8,2.1,3.1,2.5,T
9,SRK,F or R,325,2.2,2.2,4.6,3.2,T


In [7]:
data["Accegrade"] = data["Accegrade"] == "T"
data

,System,mount,price,con,snow,ice,dur,Accegrade
0,SK,F,206,1.9,1.4,1.8,2.7,False
1,SRK,F or R,520,2.1,0.8,3.8,2.3,False
2,SK,F,160,1.7,1.9,1.6,3.7,False
3,SK,F,213,1.7,2.0,2.4,3.4,False
4,SMS,F or R,598,1.6,2.4,2.7,2.8,False
5,SK,F,109,2.0,1.9,2.4,3.7,False
6,SRK,F or R,325,2.0,2.1,3.2,2.8,False
7,SMS,F or R,498,1.5,3.3,3.5,2.0,True
8,SRK,F or R,396,2.8,2.1,3.1,2.5,True
9,SRK,F or R,325,2.2,2.2,4.6,3.2,True


In [8]:
X_train = data.iloc[:-3].drop(columns=["Accegrade"])
y_train = data.iloc[:-3]["Accegrade"]
X_test = data.iloc[-3:].drop(columns=["Accegrade"])
y_test = data.iloc[-3:]["Accegrade"]

prediction = classificator(X_train, y_train, X_test, verbose=1)
print(f"\nPrediction:\n{prediction}")
print("\nAccuracy_score:\n", accuracy_score(prediction, y_test))

['SK' 'F' (99, 206) (1.9, 2.8) (1.4, 2.2) (1.8, 2.5) (2.7, 4.0)]
[(0, False), (5, False)]
['SK' 'F' (140, 206) (1.9, 2.6) (1.4, 2.3) (1.8, 3.3) (2.7, 3.4)]
[(0, False)]
['SK' 'F' (206, 215) (1.9, 2.3) (1.4, 3.8) (1.8, 4.8) (2.3, 2.7)]
[(0, False)]

Prediction:
[False, False, False]

Accuracy_score:
 0.0


In [9]:
clf = fcalc.classifier.PatternBinaryClassifier(X_train.values, y_train.to_numpy(), categorical=[1, 2], method= "ratio-support")
clf.predict(X_test.values)

print("Fcalc.classifier.PatternBinaryClassifier predicions:")
print(clf.predictions)
actual_predictions = [i == 1 for i in clf.predictions]
print("\nAccuracy_score:\n", accuracy_score(actual_predictions, y_test))

Fcalc.classifier.PatternBinaryClassifier predicions:
[ 0.  1. -1.]

Accuracy_score:
 0.3333333333333333


### Comments for results
Here we have very poor accuracy. In my opinion, this is because we have only one object in most of our extents, so we have to change our algorithm a bit, such as taking the average prediction instead of just the first one, or changing the intervals calculating function.

# Subtask 2

In [10]:
data = pd.read_csv("Wheel_Chains_Dataset_difforder.csv", sep=';')
data

,System,mount,price,con,snow,ice,dur,Accegrade
0,SK,F,149,1.9,2.5,4.0,3.8,T
1,SRK,F or R,520,2.1,0.8,3.8,2.3,F
2,SRK,F or R,389,2.0,2.2,3.3,4.3,T
3,SK,F,213,1.7,2.0,2.4,3.4,F
4,SMS,F or R,598,1.6,2.4,2.7,2.8,F
5,SK,F,109,2.0,1.9,2.4,3.7,F
6,SRK,F or R,325,2.0,2.1,3.2,2.8,F
7,SMS,F or R,498,1.5,3.3,3.5,2.0,T
8,SRK,F or R,396,2.8,2.1,3.1,2.5,T
9,SK,F,160,1.7,1.9,1.6,3.7,F


In [11]:
data["Accegrade"] = data["Accegrade"] == "T"
data

,System,mount,price,con,snow,ice,dur,Accegrade
0,SK,F,149,1.9,2.5,4.0,3.8,True
1,SRK,F or R,520,2.1,0.8,3.8,2.3,False
2,SRK,F or R,389,2.0,2.2,3.3,4.3,True
3,SK,F,213,1.7,2.0,2.4,3.4,False
4,SMS,F or R,598,1.6,2.4,2.7,2.8,False
5,SK,F,109,2.0,1.9,2.4,3.7,False
6,SRK,F or R,325,2.0,2.1,3.2,2.8,False
7,SMS,F or R,498,1.5,3.3,3.5,2.0,True
8,SRK,F or R,396,2.8,2.1,3.1,2.5,True
9,SK,F,160,1.7,1.9,1.6,3.7,False


In [12]:
X_train = data.iloc[:-3].drop(columns=["Accegrade"])
y_train = data.iloc[:-3]["Accegrade"]
X_test = data.iloc[-3:].drop(columns=["Accegrade"])
y_test = data.iloc[-3:]["Accegrade"]

prediction = classificator(X_train, y_train, X_test, verbose=1)
print(f"\nPrediction:\n{prediction}")
print("\nAccuracy_score:\n", accuracy_score(prediction, y_test))

['SK' 'F' (99, 149) (1.9, 2.8) (2.2, 2.5) (2.5, 4.0) (3.8, 4.0)]
[(0, True)]
['SK' 'F' (140, 149) (1.9, 2.6) (2.3, 2.5) (3.3, 4.0) (3.4, 3.8)]
[(0, True)]
['SK' 'F' (149, 215) (1.9, 2.3) (2.5, 3.8) (4.0, 4.8) (2.3, 3.8)]
[(0, True)]

Prediction:
[True, True, True]

Accuracy_score:
 1.0


In [13]:
clf = fcalc.classifier.PatternBinaryClassifier(X_train.values, y_train.to_numpy(), categorical=[1, 2], method= "ratio-support")
clf.predict(X_test.values)

print("Fcalc.classifier.PatternBinaryClassifier predicions:")
print(clf.predictions)
actual_predictions = [i == 1 for i in clf.predictions]
print("\nAccuracy_score:\n", accuracy_score(actual_predictions, y_test))

print("\nWe can see, that after we have changed the order of the dataset, our accuracy improved dramatically.\n"
      "That's why we should re-evaluate accuracy with he help of cross-validation.")

Fcalc.classifier.PatternBinaryClassifier predicions:
[1. 1. 1.]

Accuracy_score:
 1.0

We can see, that after we have changed the order of the dataset, our accuracy improved dramatically.
That's why we should re-evaluate accuracy with he help of cross-validation.


# Subtask 3

In [14]:
# Cross validation function for counting Lazy FCA classifier avarage accuracy.
def fcalc_cross_validation(method, data, target, folds=5, verbose=0):
    my_accuracies = np.array([])
    fcalc_accuracies = np.array([])
    fold_size = int(np.ceil(data.shape[0] / folds))

    for i in range(0, data.shape[0], fold_size):
        X_train = pd.concat([data.iloc[: i], data.iloc[i + fold_size:]])
        y_train = pd.concat([target.iloc[: i], target.iloc[i + fold_size:]])

        X_test = data.iloc[i : i + fold_size]
        y_test = target.iloc[i : i + fold_size]

        if verbose:
            print(f"Fold {i // fold_size + 1}")

        prediction = classificator(X_train, y_train, X_test, verbose=verbose)

        clf = fcalc.classifier.PatternBinaryClassifier(X_train.values, y_train.to_numpy(), categorical=[1, 2], method=method)
        clf.predict(X_test.values)
        actual_predictions = [i == 1 for i in clf.predictions]

        my_accuracy = accuracy_score(y_test, prediction)
        fcalc_accuracy = accuracy_score(y_test, actual_predictions)

        my_accuracies = np.append(my_accuracies, my_accuracy)
        fcalc_accuracies = np.append(fcalc_accuracies, fcalc_accuracy)
        if verbose:
            print("My Accuracy for fold %d is %.2f \n"%(i // fold_size + 1, my_accuracy))
            print("FCALC Accuracy for fold %d is %.2f \n"%(i // fold_size + 1, fcalc_accuracy))

    return np.average(my_accuracies), np.average(fcalc_accuracies)

In [15]:
my_accuracy, fcalc_accuracy = fcalc_cross_validation(method="ratio-support", 
                                                    data=data.drop(columns=["Accegrade"]), 
                                                    target=data["Accegrade"])

print("Our cross validation accuracy: %.2f"%(my_accuracy) + ';')
print("FCALC cross validation accuracy: %.2f"%(fcalc_accuracy) + '.')

Our cross validation accuracy: 0.65;
FCALC cross validation accuracy: 0.75.


# Subtask 4

In [16]:
def min_inf_interval(a_1, a_2):
    return (min(a_1, a_2), float('inf'))

raw_data = data.drop(columns=["Accegrade"])
target = data["Accegrade"]

X_train = pd.concat([raw_data.iloc[:14], raw_data.iloc[15:]])
y_train = pd.concat([target.iloc[:14], target.iloc[15:]])

X_test = raw_data.iloc[14:15]
y_test = target.iloc[14:15]

In [18]:
prediction = classificator(X_train, y_train, X_test, verbose=1, interval_constructor=min_inf_interval)

print("\nClassificator predicion:")
print(prediction)
print("\nAccuracy_score:\n", accuracy_score(prediction, y_test))

['SK' 'F' (99, inf) (1.9, inf) (2.2, inf) (2.5, inf) (3.8, inf)]
[(0, True)]

Classificator predicion:
[True]

Accuracy_score:
 1.0


# Subtask 5

In [19]:
import warnings
warnings.filterwarnings('ignore')

def max_inf_interval(a_1, a_2):
    return (max(a_1, a_2), float('inf'))

prediction = classificator(X_train, y_train, X_test, verbose=1, interval_constructor=max_inf_interval)
print("\nClassificator predicion:")
print(prediction)
print("\nAccuracy_score:\n", accuracy_score(prediction, y_test))

print("\nDrawing a conclusion we can say that first method is more suitable for our dataset as it isn't so strict.")

['SK' 'F' (149, inf) (2.8, inf) (2.5, inf) (4.0, inf) (4.0, inf)]
[]
['*' '*' (520, inf) (2.8, inf) (2.2, inf) (3.8, inf) (4.0, inf)]
[]
['*' '*' (389, inf) (2.8, inf) (2.2, inf) (3.3, inf) (4.3, inf)]
[]
['SK' 'F' (213, inf) (2.8, inf) (2.2, inf) (2.5, inf) (4.0, inf)]
[]
['*' '*' (598, inf) (2.8, inf) (2.4, inf) (2.7, inf) (4.0, inf)]
[]
['SK' 'F' (109, inf) (2.8, inf) (2.2, inf) (2.5, inf) (4.0, inf)]
[]
['*' '*' (325, inf) (2.8, inf) (2.2, inf) (3.2, inf) (4.0, inf)]
[]
['*' '*' (498, inf) (2.8, inf) (3.3, inf) (3.5, inf) (4.0, inf)]
[]
['*' '*' (396, inf) (2.8, inf) (2.2, inf) (3.1, inf) (4.0, inf)]
[]
['SK' 'F' (160, inf) (2.8, inf) (2.2, inf) (2.5, inf) (4.0, inf)]
[]
['*' '*' (389, inf) (2.8, inf) (2.2, inf) (3.3, inf) (4.3, inf)]
[]
['*' 'F' (298, inf) (2.8, inf) (2.3, inf) (3.3, inf) (4.0, inf)]
[]
['SK' 'F' (206, inf) (2.8, inf) (2.2, inf) (2.5, inf) (4.0, inf)]
[]
['*' '*' (684, inf) (2.8, inf) (3.3, inf) (4.4, inf) (4.0, inf)]
[]
['SK' 'F' (140, inf) (2.8, inf) (2.3, inf) 